In [2]:
def search_and_write(token, input_filepath, output_filepath, opposite):
    with open(input_filepath, 'r') as infile, open(output_filepath, 'w') as outfile:
        count = 1
        direct_coprod_info = "Herbert H. Landy Insurance!!Agency, Inc.!!100 River Ridge Drive #301!!!!Norwood!!MA!!02062!!800-336-5422"
        for line in infile:
            if opposite == False:
                if token in line.replace(direct_coprod_info, "") or line == '' or count == 1:
                    outfile.write(line)
                    count+= 1
            else:
                if token not in line.replace(direct_coprod_info, "") or line == '' or count == 1:
                    outfile.write(line)
                    count+= 1

search_and_write('!!MA!!', '/Volumes/pdf_files/win98/RE_RENEWALS/RE_Express_1022.txt', '/Volumes/pdf_files/garf/Kris/Projects/Ma0819Version_20250226/RE_Express_0819.txt', False)
search_and_write('!!MA!!', '/Volumes/pdf_files/win98/RE_RENEWALS/RE_Express_1022.txt', '/Volumes/pdf_files/garf/Kris/Projects/Ma0819Version_20250226/RE_Express_1022.txt', True)

In [3]:
import psycopg2

# Connect to your postgres DB
conn = psycopg2.connect("dbname=landy user=postgres password=postgres host=10.0.0.51")

# Open a cursor to perform database operations
cur = conn.cursor()

# Execute a query
cur.execute("select prosrc from pg_proc where oid = 16398;")

# Retrieve query results
records = cur.fetchall()

# Print the output
for record in records:
    print(record[0])

# Close communication with the database
cur.close()
conn.close()



declare

iter integer; -- dummy integer to iterate columns with
col text; -- variable to keep the column name at each iteration
col_first text; -- first column name, e.g., top left corner on a csv file or spreadsheet

begin
    set schema 'public';

    create table temp_table ();

    -- add just enough number of columns
    for iter in 1..col_count
    loop
        execute format('alter table temp_table add column col_%s text;', iter);
    end loop;

    -- copy the data from csv file
    execute format('copy temp_table from %L with delimiter ''^'' quote E''\b'' csv ', csv_path);

    iter := 1;
    col_first := (select col_1 from temp_table limit 1);

    -- update the column names based on the first row which has the column names
    for col in execute format('select unnest(string_to_array(trim(temp_table::text, ''()''), '','')) from temp_table where col_1 = %L', col_first)
    loop
        execute format('alter table temp_table rename column col_%s to "%s"', iter, col);
        